# Methodology

This chapter describes our analytical approach for estimating the budgetary impacts of Social Security benefit taxation reforms. We employ microsimulation modeling using PolicyEngine, an open-source platform that provides detailed estimates of policy changes on federal tax revenue and household-level impacts.

## Microsimulation Framework

### PolicyEngine Platform

Our analysis uses PolicyEngine US, an open-source tax-benefit microsimulation model. PolicyEngine provides several advantages for this analysis:

- **Comprehensive Tax Code Implementation**: Complete modeling of federal income tax provisions, including Social Security benefit taxation rules
- **Current Data**: Based on the most recent enhanced Current Population Survey data (2024)
- **Policy Flexibility**: Allows detailed specification of complex policy reforms
- **Transparency**: Open-source codebase enables verification and replication
- **Validation**: Regular benchmarking against official government estimates

In [1]:
# Import necessary libraries and set up environment
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, HTML
import plotly.express as px
import plotly.graph_objects as go
import os
import warnings

# Define PolicyEngine's color palette
BLACK = "#000000"
BLUE_LIGHT = "#D8E6F3"
BLUE_PRIMARY = "#2C6496"
DARK_GRAY = "#616161"
LIGHT_GRAY = "#F2F2F2"
WHITE = "#FFFFFF"

# Suppress warnings for clean output
warnings.filterwarnings('ignore')

print("PolicyEngine libraries loaded successfully!")

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PolicyEngine libraries loaded successfully!


This code cell sets up the computational environment by importing all necessary Python libraries for the microsimulation analysis. It loads PolicyEngine for tax calculations, Pandas and NumPy for data manipulation, and Plotly for visualization. The cell also defines PolicyEngine's standard color palette for consistent formatting throughout the analysis.

In [2]:
# Define reform functions for all policy options

def get_option1_reform():
    """Option 1: Full Repeal of Social Security Benefits Taxation"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option2_reform():
    """Option 2: Taxation of 85% of Social Security Benefits"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option3_reform():
    """Option 3: 85% Taxation with Permanent Senior Deduction Extension"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.senior_deduction_extension.applies": {
            "2026-01-01.2100-12-31": True
        }
    }, country_id="us")

def get_option4_reform():
    """Option 4: Social Security Tax Credit System (\$500 Credit)"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.ss_credit.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.ss_credit.amount.JOINT": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SINGLE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 500
        },
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

def get_option5_reform():
    """Option 5: Roth-Style Swap"""
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2100-12-31": 1.0
        }
    }, country_id="us")

def get_option6_reform():
    """Option 6: Phased Roth-Style Swap"""
    reform_dict = {
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2026-12-31": 0.1307,
            "2027-01-01.2027-12-31": 0.2614,
            "2028-01-01.2028-12-31": 0.3922,
            "2029-01-01.2029-12-31": 0.5229,
            "2030-01-01.2030-12-31": 0.6536,
            "2031-01-01.2031-12-31": 0.7843,
            "2032-01-01.2032-12-31": 0.9150,
            "2033-01-01.2100-12-31": 1.0
        },
        "gov.irs.social_security.taxability.rate.base": {
            "2029-01-01.2029-12-31": 0.45,
            "2030-01-01.2030-12-31": 0.40,
            "2031-01-01.2031-12-31": 0.35,
            "2032-01-01.2032-12-31": 0.30,
            "2033-01-01.2033-12-31": 0.25,
            "2034-01-01.2034-12-31": 0.20,
            "2035-01-01.2035-12-31": 0.15,
            "2036-01-01.2036-12-31": 0.10,
            "2037-01-01.2037-12-31": 0.05,
            "2038-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2029-01-01.2029-12-31": 0.80,
            "2030-01-01.2030-12-31": 0.75,
            "2031-01-01.2031-12-31": 0.70,
            "2032-01-01.2032-12-31": 0.65,
            "2033-01-01.2033-12-31": 0.60,
            "2034-01-01.2034-12-31": 0.55,
            "2035-01-01.2035-12-31": 0.50,
            "2036-01-01.2036-12-31": 0.45,
            "2037-01-01.2037-12-31": 0.40,
            "2038-01-01.2038-12-31": 0.35,
            "2039-01-01.2039-12-31": 0.30,
            "2040-01-01.2040-12-31": 0.25,
            "2041-01-01.2041-12-31": 0.20,
            "2042-01-01.2042-12-31": 0.15,
            "2043-01-01.2043-12-31": 0.10,
            "2044-01-01.2044-12-31": 0.05,
            "2045-01-01.2100-12-31": 0
        }
    }
    return Reform.from_dict(reform_dict, country_id="us")

def get_option7_reform():
    """Option 7: Eliminate Bonus Senior Deduction

    Eliminates the $6,000 bonus senior deduction from the One Big Beautiful Bill
    that has a 6% phase-out beginning at $75k/$150k for single/joint filers.
    The deduction expires in 2029, so there's only impact from 2026-2028.
    """
    return Reform.from_dict({
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        }
    }, country_id="us")

print("Policy reform functions defined successfully!")

Policy reform functions defined successfully!


This computational step pre-calculates the baseline (current law) tax liabilities for all analysis years (2026-2035). By computing these baselines once and reusing them for all policy options, we significantly improve computational efficiency while ensuring consistent comparison across all reforms.

This function generates the comprehensive policy impacts report by formatting the calculated results into markdown tables and detailed analysis. It creates both summary tables showing 10-year totals and year-by-year breakdowns, then writes the complete analysis to the `policy_impacts.md` file that forms the next chapter of this study.

In [3]:
# Pre-compute baselines for all years to avoid redundant calculations
print("Pre-computing baselines for all years...")
baselines = {}
years = list(range(2026, 2036))

for year in years:
    print(f"  Computing baseline for {year}...")
    baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    baseline_income_tax = baseline.calculate("income_tax", map_to="household", period=year)
    baselines[year] = baseline_income_tax
    
print("Baseline computation complete!\n")

Pre-computing baselines for all years...
  Computing baseline for 2026...
  Computing baseline for 2027...
  Computing baseline for 2028...
  Computing baseline for 2029...
  Computing baseline for 2030...
  Computing baseline for 2031...
  Computing baseline for 2032...
  Computing baseline for 2033...
  Computing baseline for 2034...
  Computing baseline for 2035...
Baseline computation complete!



In [4]:
# Function to calculate budgetary impact for a given reform and year
def calculate_budgetary_impact(reform, year, baseline_income_tax):
    """
    Calculate the budgetary impact of a reform for a specific year
    Returns impact in billions of dollars (positive = revenue increase, negative = revenue loss/cost)
    """
    # Create microsimulation with reform
    microsim_reform = Microsimulation(
        reform=reform,
        dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
    )
    
    # Calculate reform income tax
    reform_income_tax = microsim_reform.calculate("income_tax", map_to="household", period=year)
    
    # Calculate impact (baseline - reform)
    # Positive values mean reform reduces tax revenue (cost)
    # Negative values mean reform increases tax revenue
    # Calculate revenue change (reform - baseline)
    # Positive values mean reform increases tax revenue
    # Negative values mean reform reduces tax revenue (costs money)
    impact = (reform_income_tax.sum() - baseline_income_tax.sum()) / 1e9  # Convert to billions
    
    return impact

print("Budgetary impact calculation function defined!")

Budgetary impact calculation function defined!


In [ ]:
# Calculate impacts for each reform option
print("Calculating budgetary impacts for all policy options...\n")
results = {}

# Define all reform options
reforms = {
    "Option 1": get_option1_reform(),
    "Option 2": get_option2_reform(),
    "Option 3": get_option3_reform(),
    "Option 4": get_option4_reform(),
    "Option 5": get_option5_reform(),
    "Option 6": get_option6_reform(),
    "Option 7": get_option7_reform()
}

# Calculate impacts for each option and year
for option_name, reform in reforms.items():
    print(f"Calculating {option_name}...")
    results[option_name] = []
    
    for year in years:
        impact = calculate_budgetary_impact(reform, year, baselines[year])
        results[option_name].append(impact)
        print(f"  Year {year}: ${impact:.1f} billion")
    
    print(f"  10-year total: ${sum(results[option_name]):.1f} billion\n")

# Prepare data for display
yearly_data = {
    "Year": years,
    "Option 1": results["Option 1"],
    "Option 2": results["Option 2"],
    "Option 3": results["Option 3"],
    "Option 4": results["Option 4"],
    "Option 5": results["Option 5"],
    "Option 6": results["Option 6"],
    "Option 7": results["Option 7"]
}

# Calculate 10-year totals
totals = {option: sum(impacts) for option, impacts in results.items()}

print("All calculations complete!")

Calculating budgetary impacts for all policy options...

Calculating Option 1...
  Year 2026: $-86.3 billion
  Year 2027: $-91.8 billion
  Year 2028: $-96.9 billion
  Year 2029: $-105.9 billion
  Year 2030: $-112.7 billion
  Year 2031: $-118.4 billion
  Year 2032: $-124.5 billion
  Year 2033: $-130.7 billion
  Year 2034: $-137.5 billion
  Year 2035: $-144.3 billion
  10-year total: $-1149.1 billion

Calculating Option 2...
  Year 2026: $18.3 billion
  Year 2027: $18.3 billion
  Year 2028: $18.1 billion
  Year 2029: $23.8 billion
  Year 2030: $24.0 billion
  Year 2031: $23.9 billion
  Year 2032: $23.7 billion
  Year 2033: $23.3 billion
  Year 2034: $23.1 billion
  Year 2035: $22.8 billion
  10-year total: $219.3 billion

Calculating Option 3...
  Year 2026: $18.3 billion
  Year 2027: $18.3 billion
  Year 2028: $18.1 billion
  Year 2029: $-3.5 billion
  Year 2030: $-3.5 billion
  Year 2031: $-3.6 billion
  Year 2032: $-3.7 billion
  Year 2033: $-3.9 billion
  Year 2034: $-4.2 billion
  Y

In [ ]:
# Function to write results to policy_impacts.md
def write_results_to_markdown():
    """
    Write the calculated results to policy_impacts.md file
    """
    
    # Prepare the summary table
    summary_data = [
        {"Policy Option": "**Option 1**: Full Repeal", "10-Year Impact (Billions)": f"\\${totals['Option 1']:.1f}"},
        {"Policy Option": "**Option 2**: 85% Taxation", "10-Year Impact (Billions)": f"\\${totals['Option 2']:.1f}"},
        {"Policy Option": "**Option 3**: 85% with Senior Deduction", "10-Year Impact (Billions)": f"\\${totals['Option 3']:.1f}"},
        {"Policy Option": "**Option 4**: Tax Credit (\\$500)", "10-Year Impact (Billions)": f"\\${totals['Option 4']:.1f}"},
        {"Policy Option": "**Option 5**: Roth-Style Swap", "10-Year Impact (Billions)": f"\\${totals['Option 5']:.1f}"},
        {"Policy Option": "**Option 6**: Phased Roth-Style", "10-Year Impact (Billions)": f"\\${totals['Option 6']:.1f}"},
        {"Policy Option": "**Option 7**: Eliminate Bonus Senior Deduction", "10-Year Impact (Billions)": f"\\${totals['Option 7']:.1f}"}
    ]
    
    # Create markdown tables manually to avoid tabulate dependency issues
    
    # Summary table
    summary_table = "| Policy Option | 10-Year Impact (Billions) |\\n|---|---|\\n"
    for item in summary_data:
        summary_table += f"| {item['Policy Option']} | {item['10-Year Impact (Billions)']} |\\n"
    
    # Year-by-year table with totals
    yearly_table = "| Year | Option 1 | Option 2 | Option 3 | Option 4 | Option 5 | Option 6 | Option 7 |\\n"
    yearly_table += "|------|----------|----------|----------|----------|----------|----------|----------|\\n"
    
    # Add yearly data
    for i, year in enumerate(yearly_data['Year']):
        yearly_table += f"| {year} | \\${yearly_data['Option 1'][i]:.1f} | \\${yearly_data['Option 2'][i]:.1f} | \\${yearly_data['Option 3'][i]:.1f} | \\${yearly_data['Option 4'][i]:.1f} | \\${yearly_data['Option 5'][i]:.1f} | \\${yearly_data['Option 6'][i]:.1f} | \\${yearly_data['Option 7'][i]:.1f} |\\n"
    
    # Add totals row
    yearly_table += f"| **2026-2035 Total** | **\\${totals['Option 1']:.1f}** | **\\${totals['Option 2']:.1f}** | **\\${totals['Option 3']:.1f}** | **\\${totals['Option 4']:.1f}** | **\\${totals['Option 5']:.1f}** | **\\${totals['Option 6']:.1f}** | **\\${totals['Option 7']:.1f}** |\\n"
    
    # Determine if values are positive or negative for proper description
    def format_impact(value):
        if value < 0:
            return f"-\\${abs(value):.1f} billion"
        else:
            return f"\\${value:.1f} billion"
    
    # Write the complete markdown file
    content = f"""# Policy Impacts

This chapter presents the quantitative results of the analysis of seven Social Security benefit taxation reform options. All estimates represent changes in federal tax revenue over the 10-year period 2026-2035, measured in billions of dollars. Positive values indicate costs (deficit increasing), while negative values indicate revenue increases (deficit reducing).

## Summary of 10-Year Budgetary Impacts

The microsimulation analysis shows the following budgetary impacts across the seven policy options:

{summary_table}

## Year-by-Year Federal Budgetary Impact (2026-2035)

**Federal Budgetary Impact by Year (Billions of Dollars)**

{yearly_table}

## Detailed Results by Policy Option

### Option 1: Full Repeal of Social Security Benefits Taxation

**10-Year Impact: {format_impact(totals['Option 1'])}**

This option eliminates all federal income taxation of Social Security benefits. The revenue impact changes over time due to:
- Changes in the number of Social Security beneficiaries
- Changes in benefit levels
- Expiration of senior deduction in 2029

### Option 2: Taxation of 85% of Social Security Benefits

**10-Year Impact: {format_impact(totals['Option 2'])}**

This option taxes 85% of Social Security benefits for all recipients. The results show that expanding taxation to all beneficiaries at 85% does not offset the revenue change from eliminating income thresholds under the current senior deduction structure.

### Option 3: 85% Taxation with Permanent Senior Deduction Extension

**10-Year Impact: {format_impact(totals['Option 3'])}**

This option taxes 85% of Social Security benefits while permanently extending the senior deduction. The change after 2028 reflects the interaction between expanded benefit taxation and the permanent senior deduction.

### Option 4: Social Security Tax Credit System (\\$500 Credit)

**10-Year Impact: {format_impact(totals['Option 4'])}**

This option implements a \\$500 tax credit while expanding the taxation base to 85% of all benefits. The change after 2028 reflects the interaction between the credit system and the expiration of the senior deduction.

### Option 5: Roth-Style Swap

**10-Year Impact: {format_impact(totals['Option 5'])}**

This option replaces benefit taxation with employer payroll contribution taxation. The revenue impact changes over time as the difference between benefit taxation revenue and employer contribution taxation revenue evolves.

### Option 6: Phased Roth-Style Swap

**10-Year Impact: {format_impact(totals['Option 6'])}**

This option implements a phased transition that includes:
- Gradual reduction of benefit taxation revenue
- Gradual increase of employer contribution taxation
- Extended transition period

### Option 7: Eliminate Bonus Senior Deduction

**10-Year Impact: {format_impact(totals['Option 7'])}**

This option eliminates the \\$6,000 bonus senior deduction from the One Big Beautiful Bill starting in 2026. The revenue increase reflects:
- Elimination of the deduction for eligible taxpayers aged 65 and older
- Phase-out effects for higher-income taxpayers (6% phase-out starting at \\$75k single/\\$150k joint)
- Impact limited to 2026-2028 as the deduction expires in 2029 under current law
- Revenue pattern shows impact only in years when the deduction would have been available

## Key Findings

### Revenue Impact Ranking

From smallest to largest absolute revenue impact:
1. **Option 3**: {format_impact(totals['Option 3'])}
2. **Option 4**: {format_impact(totals['Option 4'])}
3. **Option 7**: {format_impact(totals['Option 7'])}
4. **Option 2**: {format_impact(totals['Option 2'])}
5. **Option 5**: {format_impact(totals['Option 5'])}
6. **Option 1**: {format_impact(totals['Option 1'])}
7. **Option 6**: {format_impact(totals['Option 6'])}

### Impact of Senior Deduction Expiration

The scheduled 2028 expiration of the bonus senior deduction affects the revenue patterns of several options:

- **Options 1, 5, 6**: Revenue impacts change after 2028
- **Option 2**: Revenue pattern changes after 2028
- **Option 3**: Different pattern due to permanent extension
- **Option 4**: Credit interacts with deduction expiration
- **Option 7**: Only has impact 2026-2028 since deduction expires in 2029

### Policy Design Observations

1. **Tax Credit**: The \\$500 tax credit option combines a credit with expanded benefit taxation.

2. **Roth-Style Swap**: The analysis shows different revenue impacts between benefit taxation and employer payroll contribution taxation.

3. **Phase-in Effects**: Gradual implementation (Option 6) produces different revenue patterns than immediate implementation (Option 5).

4. **Senior Deduction**: The permanent senior deduction extension (Option 3) affects the net revenue impact of expanded benefit taxation.

5. **Bonus Deduction Elimination**: Option 7 provides revenue only during the years the deduction would have been in effect (2026-2028).

## Distributional Considerations

The analysis focuses on aggregate budgetary impacts. The distributional effects vary by option:

- **Option 1**: Affects all current Social Security recipients
- **Options 2 & 3**: Changes tax treatment for beneficiaries currently below thresholds
- **Option 4**: Implements a credit for Social Security recipients
- **Options 5 & 6**: Changes the tax treatment between retirees and current workers
- **Option 7**: Affects elderly taxpayers eligible for the bonus senior deduction, with phase-out for higher incomes

## Methodology Notes

These estimates are based on static microsimulation analysis and do not account for:
- Behavioral responses to tax changes
- Economic growth effects
- Administrative implementation costs
- State tax interactions

The results are presented for comparison with estimates from other analytical sources in the next chapter."""
    
    # Write to file
    with open('../policy_impacts.md', 'w') as f:
        f.write(content)
    
    print("Results successfully written to policy_impacts.md")
    return content

# Execute the function to write results
markdown_content = write_results_to_markdown()

## Data Foundation

### Enhanced Current Population Survey

The microsimulation is based on the Enhanced Current Population Survey (CPS) for 2024:

- **Sample Size**: Approximately 160,000 individuals in 60,000 households
- **Representative Coverage**: Nationally representative sample weighted to match U.S. population
- **Income Sources**: Detailed information on wages, Social Security benefits, pensions, and other income
- **Demographics**: Age, filing status, state of residence, and other relevant characteristics
- **Enhancement**: Survey data enhanced with statistical matching to capture income and benefit details

### Tax Calculator Engine

PolicyEngine's tax calculator implements current law provisions including:

- Federal income tax brackets and rates
- Standard and itemized deductions
- Social Security benefit taxation rules (current two-tier system)
- Senior deduction provisions
- Credits and other tax preferences
- Alternative Minimum Tax calculations